In [ ]:
!pip install -q accelerate==0.21.0 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
!pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q transformers==4.31.0 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off

In [ ]:
!pip install datasets

In [ ]:
!pip install -q wandb

In [ ]:
import wandb
wandb.login()
%env WANDB_PROJECT=Lama2-fine-tuning_V1_4

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ravihansa-lk (ravihansa-lk-unitec-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=Lama2-fine-tuning_V1_4


In [ ]:
!pip uninstall -y bitsandbytes
!pip install --no-cache-dir bitsandbytes

Found existing installation: bitsandbytes 0.40.2
Uninstalling bitsandbytes-0.40.2:
  Successfully uninstalled bitsandbytes-0.40.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 300.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade bitsandbytes

In [ ]:
!pip install --upgrade nvidia-pyindex
!pip install --upgrade nvidia-cuda-runtime-cu12
!pip install --upgrade nvidia-cublas-cu12
!pip install --upgrade nvidia-cuda-nvrtc-cu12

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8419 sha256=c3a6158e10bec1c1114e422a045a303cf70e85ec46ea9a0230d9215fa64386b6
  Stored in directory: /root/.cache/pip/wheels/49/d0/7d/b68b3665d16ee20355e65fb7ef48b7ca26533217d9f09924fe
Successfully built nvidia-pyindex
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 48.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cuda-runtime-cu12
    Found existing installation: nvidia-cuda-runtime-cu12 12.4.127
    Uninstalling nvidia-cuda-runtime-cu12-12.4.127:
      Successfully uninstalled nvidia-cuda-runtime-cu12-12.4.127
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12

In [ ]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config



def load_model(model_name, bnb_config):



    n_gpus = torch.cuda.device_count()


    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
       # max_memory = {i: max_memory for i in range(n_gpus)},
    )


    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = True)


    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer


from huggingface_hub import login

# Replace 'your_token' with your actual token
login(token='')

model_name = "meta-llama/Llama-2-7b-hf"

load_in_4bit = True


bnb_4bit_use_double_quant = True


bnb_4bit_quant_type = "nf4"


bnb_4bit_compute_dtype = torch.bfloat16




bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

model, tokenizer = load_model(model_name, bnb_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
eval_prompt = """You are a helpful and patient math tutor chatbot. Your task is to assist users with their math problems by responding in a supportive

### Instruction:
Respond to the given user query in a way that encourages learning and understanding"

Input:
I'm not sure why we need to find a common denominator.

### Response : """


model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    output = model.generate(
        **model_input,
        max_new_tokens=500,
        num_beams=5,
        early_stopping=True,
        temperature=0.1
    )
    result = tokenizer.decode(output[0], skip_special_tokens=True)
print(result)

In [ ]:
dataset_name = "/content/drive/My Drive/Chat_Reserch/chat/cleaned_dataset_4_29.csv"

# Load dataset
dataset = load_dataset("csv", data_files = dataset_name, split = "train")
print(dataset[0])
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Generating train split: 0 examples [00:00, ? examples/s]

{'instruction': "Respond to the student's question in a unique and helpful way", 'Input': 'I feel nervous when I look at math problems.', 'Output': 'Take it one at a time and break it down.'}
Number of prompts: 257
Column names are: ['instruction', 'Input', 'Output']


In [ ]:
print(dataset[1])

{'instruction': "Respond to the student's question in a unique and helpful way", 'Input': 'Math always stresses me out.', 'Output': 'Break it down one part at a time.'}


In [ ]:
#split test and save it to CSV
train_test_split = dataset.train_test_split(test_size=0.2)
test_df = train_test_split['test'].to_pandas()
test_df.to_csv("/content/drive/My Drive/Chat_Reserch/chat/Test_4_29_V14.csv", index=False)

dataset = train_test_split['train']

In [ ]:
def create_prompt_formats(sample):


    INTRO_BLURB = "You are a friendly and thoughtful math tutor chatbot. Your job is to help students with math problems. Make each reply sound a little different, like a real tutor chatting with a student. Keep it short and supportive"
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"


    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['Input']}" if sample["Input"] else None
    response = f"{RESPONSE_KEY}\n{sample['Output']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]


    formatted_prompt = "\n\n".join(parts)


    sample["text"] = formatted_prompt

    return sample

In [ ]:
def get_max_length(model):


    conf = model.config

    max_length = None

    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break

    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [ ]:
def preprocess_batch(batch, tokenizer, max_length):

    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

In [ ]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):


    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)


    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ["instruction", "Input", "Output", "text"],
    )


    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)


    dataset = dataset.shuffle(seed = seed)

    return dataset

In [ ]:
print(f'Number of prompts: {len(dataset)}')

Number of prompts: 205


In [ ]:
seed = 14

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

print(preprocessed_dataset)

Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/205 [00:00<?, ? examples/s]

Map:   0%|          | 0/205 [00:00<?, ? examples/s]

Filter:   0%|          | 0/205 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 205
})


In [ ]:
print(preprocessed_dataset)
print(preprocessed_dataset[0])
print(len(preprocessed_dataset))

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 205
})
{'input_ids': [1, 887, 526, 263, 19780, 322, 2714, 1319, 5844, 260, 3406, 13563, 7451, 29889, 3575, 4982, 338, 304, 1371, 8041, 411, 5844, 4828, 29889, 8561, 1269, 8908, 6047, 263, 2217, 1422, 29892, 763, 263, 1855, 260, 3406, 521, 23980, 411, 263, 8368, 29889, 19152, 372, 3273, 322, 2304, 573, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 1666, 2818, 304, 278, 8368, 29915, 29879, 1139, 297, 263, 5412, 322, 8444, 982, 13, 13, 4290, 29901, 13, 29902, 2360, 8341, 5844, 6987, 373, 931, 29889, 13, 13, 2277, 29937, 13291, 29901, 13, 855, 388, 21732, 29892, 2867, 372, 1623, 760, 491, 760, 29889, 13, 13, 2277, 29937, 2796], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):

    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

In [ ]:
def find_all_linear_names(model):


    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

In [ ]:
def print_trainable_parameters(model, use_4bit = False):

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

In [ ]:
def fine_tune(model,
          tokenizer,
          dataset,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim,
          num_train_epochs
          ):


    model.gradient_checkpointing_enable()

    model = prepare_model_for_kbit_training(model)

    target_modules = find_all_linear_names(model)

    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    print_trainable_parameters(model)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = logits.argmax(axis=-1)
        accuracy = (predictions == labels).mean()
        return {"accuracy": accuracy}

    trainer = Trainer(
        model = model,
        train_dataset = dataset,
        args = TrainingArguments(
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
            num_train_epochs = num_train_epochs,
            report_to="wandb",
        ),
        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
    )

    model.config.use_cache = False

    do_train = True

    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok = True)
    #trainer.model.save_pretrained(output_dir)
    model.save_pretrained(output_dir)

    #del model
    #del trainer
    #torch.cuda.empty_cache()

In [ ]:

lora_r = 16

lora_alpha = 64


lora_dropout = 0.1

bias = "none"

task_type = "CAUSAL_LM"


output_dir = "./results"

per_device_train_batch_size = 2

gradient_accumulation_steps = 1

learning_rate = 2e-4

optim = "paged_adamw_32bit"

max_steps = -1

warmup_steps = 2

fp16 = True

logging_steps = 1

per_device_eval_batch_size = 2

num_train_epochs = 2

In [ ]:
#train_test_split = preprocessed_dataset.train_test_split(test_size=0.0)
train_dataset = preprocessed_dataset
#train_dataset = train_test_split['train']
#eval_dataset = train_test_split['test']
#val_df = train_test_split['test'].to_pandas()
#val_df.to_csv("Lama_clasification_LLM_val_V1_4.csv", index=False)
print(len(train_dataset))
#print(len(eval_dataset))

205


In [ ]:
fine_tune(model, tokenizer, train_dataset, lora_r, lora_alpha, lora_dropout, bias, task_type, per_device_train_batch_size, gradient_accumulation_steps, warmup_steps, max_steps, learning_rate, fp16, logging_steps, output_dir, optim,num_train_epochs)

LoRA module names: ['k_proj', 'gate_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj', 'down_proj']
All Parameters: 3,540,389,888 || Trainable Parameters: 39,976,960 || Trainable Parameters %: 1.1291682911958425
Training...


/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:427: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:1301: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  new_forward = torch.cuda.amp.autocast(dtype=torch.float16)(model_forward_func)


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.743100
2,2.763300
3,2.447000
4,1.748400
5,1.247100
6,1.441100
7,0.947500
8,0.805000
9,0.597700
10,0.500100


***** train metrics *****
  epoch                    =        2.0
  total_flos               =   866898GF
  train_loss               =     0.3174
  train_runtime            = 0:04:20.98
  train_samples_per_second =      1.571
  train_steps_per_second   =      0.789
{'train_runtime': 260.9818, 'train_samples_per_second': 1.571, 'train_steps_per_second': 0.789, 'total_flos': 930825607053312.0, 'train_loss': 0.3174469120872831, 'epoch': 2.0}
Saving last checkpoint of the model...


In [ ]:
#wandb.finish()

#model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map = "auto", torch_dtype = torch.bfloat16)
model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    offload_folder="./offload"
)

model = model.merge_and_unload()


output_merged_dir = "results/news_classification_llama2_7b/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok = True)
model.save_pretrained(output_merged_dir, safe_serialization = True)


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(output_merged_dir)

new_model = "Finetune-llama-2-7b_V32"

In [ ]:

login(token='')
model.push_to_hub(new_model, use_auth_token = True)
tokenizer.push_to_hub(new_model, use_auth_token = True)

In [ ]:
import json

# Load JSON data from files
with open('results/all_results.json') as f:
    all_results = json.load(f)

with open('results/train_results.json') as f:
    train_results = json.load(f)

with open('results/trainer_state.json') as f:
    trainer_state = json.load(f)
print(train_results)
print(all_results)
print(trainer_state)

{'epoch': 2.0, 'total_flos': 932298432380928.0, 'train_loss': 0.3108996497195901, 'train_runtime': 395.3421, 'train_samples_per_second': 1.037, 'train_steps_per_second': 0.521}
{'epoch': 2.0, 'total_flos': 932298432380928.0, 'train_loss': 0.3108996497195901, 'train_runtime': 395.3421, 'train_samples_per_second': 1.037, 'train_steps_per_second': 0.521}
{'best_metric': None, 'best_model_checkpoint': None, 'epoch': 2.0, 'global_step': 206, 'is_hyper_param_search': False, 'is_local_process_zero': True, 'is_world_process_zero': True, 'log_history': [{'epoch': 0.01, 'learning_rate': 0.0001, 'loss': 2.6148, 'step': 1}, {'epoch': 0.02, 'learning_rate': 0.0002, 'loss': 2.7816, 'step': 2}, {'epoch': 0.03, 'learning_rate': 0.00019901960784313727, 'loss': 2.3434, 'step': 3}, {'epoch': 0.04, 'learning_rate': 0.00019803921568627454, 'loss': 1.8257, 'step': 4}, {'epoch': 0.05, 'learning_rate': 0.00019705882352941177, 'loss': 1.3943, 'step': 5}, {'epoch': 0.06, 'learning_rate': 0.000196078431372549, '

In [ ]:
import json
import matplotlib.pyplot as plt

# Load JSON data from files
with open('results/all_results.json') as f:
    all_results = json.load(f)

with open('results/train_results.json') as f:
    train_results = json.load(f)

with open('results/trainer_state.json') as f:
    trainer_state = json.load(f)

# Inspect the log_history structure
log_history = trainer_state.get('log_history', [])
if not log_history:
    print("No log history available.")
else:
    print("Log History Sample Entry:")
    print(log_history[0])  # Print a sample entry for inspection

# Extract data for plotting
steps = []
loss = []
epochs = []
learning_rates = []

for entry in log_history:
    # Use .get() to avoid KeyError
    steps.append(entry.get('step', None))
    loss.append(entry.get('loss', None))
    epochs.append(entry.get('epoch', None))
    learning_rates.append(entry.get('learning_rate', None))

# Check if the lists are empty or contain None values
if not all(x is not None for x in [steps, loss, epochs, learning_rates]):
    print("Warning: Some data lists contain None values. Check your JSON structure.")
else:
    # Create a figure with subplots
    fig, axs = plt.subplots(3, 1, figsize=(12, 18))

    # Plot 1: Training Loss Over Steps
    axs[0].plot(steps, loss, marker='o', linestyle='-', color='b')
    axs[0].set_xlabel('Step')
    axs[0].set_ylabel('Loss')
    axs[0].set_title('Training Loss Over Steps')
    axs[0].grid(True)

    # Plot 2: Learning Rate Over Epochs
    axs[1].plot(epochs, learning_rates, marker='o', linestyle='-', color='r')
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('Learning Rate')
    axs[1].set_title('Learning Rate Over Epochs')
    axs[1].grid(True)

    # Plot 3: Training Loss and Metrics Over Epochs
    ax2 = axs[2].twinx()
    axs[2].plot(epochs, loss, marker='o', linestyle='-', color='b', label='Train Loss')
    ax2.plot(epochs, learning_rates, marker='o', linestyle='--', color='r', label='Learning Rate')

    axs[2].set_xlabel('Epoch')
    axs[2].set_ylabel('Train Loss', color='b')
    ax2.set_ylabel('Learning Rate', color='r')

    axs[2].set_title('Train Loss and Learning Rate Over Epochs')
    axs[2].grid(True)
    axs[2].legend(loc='upper left')
    ax2.legend(loc='upper right')

    # Show the plots
    plt.tight_layout()
    plt.show()


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=113c66ab8510bf03435a6b7cbb58c25afa562f37e166f5d7d2f55025fe05cd79
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download punkt_tab data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
dataset_name = "/content/drive/My Drive/Chat_Reserch/chat/Test_4_29_V14.csv"

# Load dataset
dataset = load_dataset("csv", data_files = dataset_name, split = "train")
print(dataset[0])
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')
#dataset = datasetAll.select(range(50))
print(f'Number of prompts: {len(dataset)}')

In [ ]:
from sklearn.metrics import f1_score
import re
from nltk.tokenize import word_tokenize
test_dataset = dataset

true_labels = []
predictions = []
data_records = []

for i in range(50):
    code_snippet = test_dataset[i]["Input"]
    true_label = test_dataset[i]["Output"]


    prompt = f"""You are a friendly and thoughtful math tutor chatbot. Your job is to help students with math problems. Make each reply sound unique, like a real tutor chatting with a student. Keep it short and supportive.

    ### Instruction:
    Respond to the student's question in a unique and helpful way

    Input:
    {code_snippet}

    ### Response:"""
    #print(prompt)
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    model.eval()
    with torch.no_grad():
      output = model.generate(
         **model_input,
          max_new_tokens=100,
          num_beams=5,
          early_stopping=True,
          temperature=0.1
    )
    result = tokenizer.decode(output[0], skip_special_tokens=True)
    #print(result)
    #print("**************************")
    #response = result.split("### Response:")[-1].strip()
    match = re.search(r"### Response:\s*(.*?)(?=\n###)", result, re.DOTALL)
    if match is None:
      match = re.search(r"### Response:\s*(.*?)(?=\n### End Response:)", result, re.DOTALL)
    response = match.group(1).strip() if match else ""
    predictions.append(response)
    true_labels.append(true_label)
    print("Model responce:"+response)
    print("true_label :"+true_label)
    rouge_score = scorer.score(true_label,response)
    #print("ROUGE-1: " f"{rouge_score['rouge1'].fmeasure:.2f}")
    #print("ROUGE-2: " f"{rouge_score['rouge2'].fmeasure:.2f}")
    #print("ROUGE-L: " f"{rouge_score['rougeL'].fmeasure:.2f}")
    #print("Input: "+code_snippet)
    #print("Responce: "+response)
    #print("Datasetoutput: "+true_label)
    print(i)
    #print(result)
    #print("***********")
    Expected_Response = word_tokenize(true_label)
    response1 = word_tokenize(response)
    blue_score = sentence_bleu([Expected_Response], response1)
    #print("BLEU Score: " f"{blue_score:.2f}")

    data_records.append({
        "User_Input": code_snippet,
        "ModelResponse": response,
        "DatasetResponse": true_label,
        "ROUGE-1": f"{rouge_score['rouge1'].fmeasure:.2f}",
        "ROUGE-2": f"{rouge_score['rouge2'].fmeasure:.2f}",
        "ROUGE-L": f"{rouge_score['rougeL'].fmeasure:.2f}",
        "BLEU Score":  f"{blue_score:.2f}",
    })

import pandas as pd
df = pd.DataFrame(data_records)
df.to_csv("/content/drive/My Drive/Chat_Reserch/chat/Finetune-llama-ResultV1_4.csv", index=False)
print("CSV file 'responses.csv' saved successfully!")